In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


The dataset used for this lab was obtained from:

Manu Siddhartha, November 6, 2020, "Bondora Peer-to-Peer Lending Data", IEEE Dataport, doi: https://dx.doi.org/10.21227/33kz-0s65.

#### Load and observe the data set


In [3]:

data = pd.read_csv('../data/loan_data.csv',low_memory=False )
data.head()


,new_customer,income_verification,language,age,gender,country,amount,loan_duration,use_of_loan,education,...,credit_score_equifax,credit_score_ee_mini,nr_previous_loans,amount_previous_loans,previous_repayments,previous_early_repayments,previous_early_repayments_count,default,credit_score_fi_asiakas_c,credit_score_fi_asiakas_n
0,0,expenses_verified,estonian,23,male,EE,5500.0,60,loan_consolidation,secondary,...,NaN,1000.0,1,3000.0,571.73,0.0,0,0,NaN,NaN
1,0,expenses_verified,russian,25,male,EE,1200.0,60,other,vocational,...,NaN,1000.0,1,500.0,194.98,0.0,0,0,NaN,NaN
2,1,unverified,estonian,22,female,EE,1000.0,60,other,basic,...,NaN,700.0,0,0.0,0.00,0.0,0,1,NaN,NaN
3,1,unverified,finnish,49,female,FI,1000.0,60,other,basic,...,NaN,NaN,0,0.0,0.00,0.0,0,0,RL2,NaN
4,0,expenses_verified,estonian,25,female,EE,4100.0,60,loan_consolidation,secondary,...,NaN,1000.0,1,1500.0,1309.03,0.0,0,0,NaN,NaN


In [4]:
data.shape

(75693, 33)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75693 entries, 0 to 75692
Data columns (total 33 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   new_customer                     75693 non-null  int64  
 1   income_verification              75693 non-null  object 
 2   language                         75693 non-null  object 
 3   age                              75693 non-null  int64  
 4   gender                           75693 non-null  object 
 5   country                          75693 non-null  object 
 6   amount                           75693 non-null  float64
 7   loan_duration                    75693 non-null  int64  
 8   use_of_loan                      75693 non-null  object 
 9   education                        75693 non-null  object 
 10  marital_status                   75693 non-null  object 
 11  nr_dependants                    30677 non-null  float64
 12  employment_status 

##### Split the data

In [8]:
missing_percentage = data.isnull().mean() * 100
missing_percentage

new_customer                        0.000000
income_verification                 0.000000
language                            0.000000
age                                 0.000000
gender                              0.000000
country                             0.000000
amount                              0.000000
loan_duration                       0.000000
use_of_loan                         0.000000
education                           0.000000
marital_status                      0.000000
nr_dependants                      59.471814
employment_status                   0.046239
employment_duration                 0.490138
work_experience                    58.369995
occupation                          0.000000
home_ownership                      0.000000
income                              0.000000
existing_debt                       0.000000
total_debt                          0.000000
debt_to_income                      0.000000
disposable_income                   0.000000
credit_sco

In [38]:
missing_columns = data.columns[100*(data.isnull().sum()/len(data.index)) > 40]
missing_columns

Index(['ContractEndDate', 'NrOfDependants', 'EmploymentPosition',
       'WorkExperience', 'PlannedPrincipalTillDate', 'CurrentDebtDaysPrimary',
       'DebtOccuredOn', 'CurrentDebtDaysSecondary',
       'DebtOccuredOnForSecondary', 'DefaultDate',
       'PlannedPrincipalPostDefault', 'PlannedInterestPostDefault', 'EAD1',
       'EAD2', 'PrincipalRecovery', 'InterestRecovery', 'RecoveryStage',
       'EL_V0', 'Rating_V0', 'EL_V1', 'Rating_V1', 'Rating_V2',
       'ActiveLateCategory', 'CreditScoreEsEquifaxRisk',
       'CreditScoreFiAsiakasTietoRiskGrade', 'CreditScoreEeMini',
       'PrincipalWriteOffs', 'InterestAndPenaltyWriteOffs',
       'PreviousEarlyRepaymentsBefoleLoan', 'GracePeriodStart',
       'GracePeriodEnd', 'NextPaymentDate', 'ReScheduledOn',
       'PrincipalDebtServicingCost', 'InterestAndPenaltyDebtServicingCost',
       'ActiveLateLastPaymentCategory'],
      dtype='object')

In [39]:
lose_data = ['ContractEndDate', 'NrOfDependants', 'EmploymentPosition',
       'WorkExperience', 'PlannedPrincipalTillDate', 'CurrentDebtDaysPrimary',
       'DebtOccuredOn', 'CurrentDebtDaysSecondary',
       'DebtOccuredOnForSecondary', 'DefaultDate',
       'PlannedPrincipalPostDefault', 'PlannedInterestPostDefault', 'EAD1',
       'EAD2', 'PrincipalRecovery', 'InterestRecovery', 'RecoveryStage',
       'EL_V0', 'Rating_V0', 'EL_V1', 'Rating_V1', 'Rating_V2',
       'ActiveLateCategory', 'CreditScoreEsEquifaxRisk',
       'CreditScoreFiAsiakasTietoRiskGrade', 'CreditScoreEeMini',
       'PrincipalWriteOffs', 'InterestAndPenaltyWriteOffs',
       'PreviousEarlyRepaymentsBefoleLoan', 'GracePeriodStart',
       'GracePeriodEnd', 'NextPaymentDate', 'ReScheduledOn',
       'PrincipalDebtServicingCost', 'InterestAndPenaltyDebtServicingCost',
       'ActiveLateLastPaymentCategory']


In [40]:
data.shape

(134529, 112)

In [41]:
new_df = data.drop(lose_data, axis=1)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134529 entries, 0 to 134528
Data columns (total 76 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   ReportAsOfEOD                           134529 non-null  object 
 1   LoanId                                  134529 non-null  object 
 2   LoanNumber                              134529 non-null  int64  
 3   ListedOnUTC                             134529 non-null  object 
 4   BiddingStartedOn                        134529 non-null  object 
 5   BidsPortfolioManager                    134529 non-null  int64  
 6   BidsApi                                 134529 non-null  int64  
 7   BidsManual                              134529 non-null  float64
 8   UserName                                134529 non-null  object 
 9   NewCreditCustomer                       134529 non-null  bool   
 10  LoanApplicationStartedDate              1345

In [ ]:

# Handle missing values
data.dropna(inplace=True)

# Encode categorical variables
label_encoders = {}
for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

# Split data into features and target variable
X = data.drop(columns=['target_variable'])
y = data['target_variable']


In [ ]:
# Scale numerical features
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
